In [14]:
import random

random.seed(42)
import numpy as np
import pandas as pd


rating = pd.read_csv("rating.csv")
rating = rating.sample(10000)
rating.describe()

movie = pd.read_csv("movie.csv")
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
movie = movie[movie['movieId'].isin(rating['movieId'].unique())]
genres_encoded = movie['genres'].str.get_dummies(sep='|')
movie_code = pd.concat([movie, genres_encoded], axis=1)
movie_code = movie_code.drop(['title', 'genres'], axis=1)
movie_code = movie_code.sum()[1:]

In [22]:
movie_code_df = pd.DataFrame(movie_code)

In [25]:
import plotly.offline as py
from plotly import graph_objs as go

data = [go.Bar(
            x = movie_code_df.index,
            y = movie_code_df.iloc[:, 0].values,
            marker= dict(colorscale='Jet',
                         color = movie_code_df.iloc[:, 0].values
                        ),
    )]

layout = go.Layout(
    title='The number of movies in different genres'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

In [26]:
rating

,userId,movieId,rating,timestamp
16000552,110728,7153,4.5,2011-04-18 05:46:14
18442827,127676,107,3.0,1998-02-02 20:54:07
10937952,75647,647,1.0,1999-12-15 08:16:30
1028265,6968,4675,3.0,2001-08-20 17:14:39
6962457,47986,10,3.0,2005-06-07 18:05:55
...,...,...,...,...
18475109,127900,4262,3.5,2008-02-01 16:40:26
7612177,52457,64030,3.0,2009-04-19 11:49:26
573610,3858,5712,2.5,2010-02-22 15:30:37
5194913,35580,59037,1.5,2009-02-15 20:52:05


In [28]:
data = [go.Bar(
            x = rating['rating'].unique(),
            y = rating['rating'].value_counts().values,
            marker= dict(colorscale='Jet',
                         color = rating['rating'].value_counts().values
                        ),
    )]

layout = go.Layout(
    title='The histogram of review star from 1 to 5'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

Action          716
Adventure       532
Animation       163
Children        253
Comedy         1260
Crime           442
Documentary      88
Drama          1629
Fantasy         284
Film-Noir        37
Horror          301
IMAX             71
Musical         147
Mystery         250
Romance         625
Sci-Fi          404
Thriller        798
War             166
Western          66
dtype: int64

In [29]:
merged = pd.merge(rating[['userId', 'movieId']], movie, on='movieId')
merged = merged.drop("movieId", axis=1)
merged.head()

,userId,title,genres
0,110728,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
1,125921,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
2,2771,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
3,79059,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
4,26294,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy


In [35]:
movie = pd.read_csv("movie.csv")

movie = movie[movie['movieId'].isin(rating['movieId'].unique())]
genres_encoded = movie['genres'].str.get_dummies(sep='|')

In [44]:
genres_encoded = genres_encoded.rename_axis('movieId')

In [46]:
merged = pd.merge(rating[['userId', 'rating', 'movieId']], genres_encoded, on='movieId')

In [55]:
columns_to_multiply = list(genres_encoded.columns)

# Multiply each selected column by the values in the multiplier column
for column in columns_to_multiply:
    merged[column] = merged[column] * merged['rating']

In [60]:
merged.replace(0, np.nan, inplace=True)

# Calculate the mean of each column considering only non-zero values
column_means = merged.mean()[3:]
column_means_df = pd.DataFrame(column_means)

In [62]:
data = [go.Bar(
            x = column_means_df.index,
            y = column_means_df.iloc[:, 0].values,
            marker= dict(colorscale='Jet',
                         color = column_means_df.iloc[:, 0].values
                        ),
    )]

layout = go.Layout(
    title='The average ratings in different genres'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')